In [2]:
import plotly.graph_objs as go
import pandas as pd

In [3]:
arbeidsmarkt = pd.read_csv("../data/arbeidsmarkt.csv", sep=';')
uurloon = pd.read_csv("../data/uurloon.csv", sep=';')
uurloon['Uurloon werknemers na verlaten ho (euro)']  = pd.to_numeric(uurloon['Uurloon werknemers na verlaten ho (euro)'] , errors='coerce')
arbeidsmarkt['Uitstromers ho (aantal)']  = pd.to_numeric(arbeidsmarkt['Uitstromers ho (aantal)'] , errors='coerce')
arbeidsmarkt.fillna(0, inplace = True)
peilmoment1 = arbeidsmarkt.loc[arbeidsmarkt['Peilmoment'] == 'Direct na verlaten onderwijs']

In [4]:
total_counts_niveau = peilmoment1.groupby('Uitstromers ho met en zonder diploma')['Uitstromers ho (aantal)'].sum().reset_index()
total_counts_niveau.rename(columns={'Uitstromers ho (aantal)': 'Total degree level'}, inplace=True)

arbeidsmarkt = arbeidsmarkt.merge(total_counts_niveau, on='Uitstromers ho met en zonder diploma')

total_counts = peilmoment1.groupby('Studierichting')['Uitstromers ho (aantal)'].sum().reset_index()
total_counts.rename(columns={'Uitstromers ho (aantal)': 'Total studierichting'}, inplace=True)

arbeidsmarkt = arbeidsmarkt.merge(total_counts, on='Studierichting')

arbeidsmarkt['Percentage studierichting'] = arbeidsmarkt['Uitstromers ho (aantal)'] / arbeidsmarkt['Total studierichting'] * 100
arbeidsmarkt['Percentage degree level'] = arbeidsmarkt['Uitstromers ho (aantal)'] / arbeidsmarkt['Total degree level'] * 100

arbeidsmarktp0 = arbeidsmarkt.loc[arbeidsmarkt['Peilmoment'] == 'Direct na verlaten onderwijs']
arbeidsmarktp1 = arbeidsmarkt.loc[arbeidsmarkt['Peilmoment'] == '1 jaar na verlaten onderwijs']
arbeidsmarktp2 = arbeidsmarkt.loc[arbeidsmarkt['Peilmoment'] == '2 jaar na verlaten onderwijs']
arbeidsmarktp3 = arbeidsmarkt.loc[arbeidsmarkt['Peilmoment'] == '3 jaar na verlaten onderwijs']
arbeidsmarktp4 = arbeidsmarkt.loc[arbeidsmarkt['Peilmoment'] == '4 jaar na verlaten onderwijs']
arbeidsmarktp5 = arbeidsmarkt.loc[arbeidsmarkt['Peilmoment'] == '5 jaar na verlaten onderwijs']

data = [arbeidsmarktp0, arbeidsmarktp1, arbeidsmarktp2, arbeidsmarktp3, arbeidsmarktp4, arbeidsmarktp5]

In [48]:
fig = go.Figure()

studies = ['']

for i in data:
    heatmap_data = i.pivot_table(values='Percentage degree level', index='Arbeidsmarktpositie', columns='Uitstromers ho met en zonder diploma', aggfunc='sum')
    heatmap_annotations = heatmap_data.map(lambda x: '{:.0f}%'.format(x))
    tracey = go.Heatmap(
        z=heatmap_data.values,
        x=heatmap_data.columns.tolist(),
        y=heatmap_data.index.tolist(),
        colorscale='Blues',
        zmin=0,
        zmax=100,
        visible=False,
        colorbar=dict(
            title='% within degree',
            tickformat='%d%',
        ),
        hoverongaps=False,
        hoverinfo='text+x',
        text=heatmap_annotations.values.tolist()
    )
    fig.add_trace(tracey)

fig.data[0].visible = True

layout = go.Layout(
    title='Labor market position per degree<br><sup>Wo-bachelor students have the most variance in labor market position throughout the years</sup>',
    xaxis=go.layout.XAxis(
        title='Degree',
        linewidth=2,
        showline=True,
        linecolor='#7785A9',
        mirror=True
    ),
    yaxis=go.layout.YAxis(
        title='Labor market position',
        linewidth=2,
        showline=True,
        linecolor='#7785A9',
        mirror=True,
        tickvals = heatmap_data.index.tolist(),
        ticktext=['✔ job, ✔ benefits', '✔ job, ✗ benefits',  'Back to university', '✗ job, ✔ benefits', '✗ job, ✗ benefits']
    )
)

steps = []
for i in range(len(fig.data)):
    step = dict(
        method="restyle",
        args=["visible", [False] * len(fig.data)],
        label=str(i),
    )
    step["args"][1][i] = True
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Years after leaving university: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders,
    height=500,
    annotations=[
        go.layout.Annotation(
            text=" <br>The heatmap illustrates the labor market position across various degrees.<br>The colors correspond to the percentages within each degree, providing a visual representation<br> of the distribution of labor market positions for graduates with different levels of education.",
            xref="paper", yref="paper",
            x=0, y=-0.8,
            showarrow=False,
            font=dict(size=12),
            xanchor='left',
            yanchor='bottom',
            align='left'
        )
    ],
    margin=dict(
        b = 160
    ),
)

fig.update_layout(layout)

fig.update_layout(margin=dict(l=20, r=20, t=80, b=10))
fig['layout']['sliders'][0]['pad']=dict(r=10, t=50, b=60)

fig.show()